In [1]:
import pandas as pd
import numpy as np
import time
import resource
from os import walk
from datetime  import datetime,timedelta,date
import multiprocessing as mp
import os
import re
import subprocess
import matplotlib.pyplot as plt
# from tld import get_tld
import shutil
import csv
import seaborn as sns
import pickle
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori

def save_obj(obj, file_name):    
    file_path = 'pyobj/' + file_name
    file_save = open(file_path, 'wb')
    pickle.dump(obj, file_save, pickle.HIGHEST_PROTOCOL)
    file_save.close()

def load_obj(file_name):
    path = 'pyobj/' + file_name
    file_save = open(path, 'rb')
    return pickle.load(file_save)

In [2]:
def read_data_file(f):
    s = open(f, 'r').read().split('|')
    song_type = s[0]
    song_data = s[1]
    
    return (song_type, song_data)

In [3]:
types = []

f = os.walk('/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song')
for fi in f:
    f = fi
    break
for fi in f[2]:
    try:
        types.append(read_data_file(f[0]+'/'+fi)[0])
    except:
        print (f[0]+'/'+fi)

/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/em-la-ba-noi-cua-anh-super-version-tang-nhat-tue.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/tinh-doi-2-chien-thang.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/last-of-the-mohicans-beautiful-song-va.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/thong_bao_loi?utm_medium=NoSong&u_source=NCT&utm_campaign=All.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/vo-tuong-lai-cao-tung-anh.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/ta-buon-mot-doi-le-tin-huong-vu-khanh.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/con-kenh-ta-dao-thanh-hoa-ft-ngoc-tan.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/goi-do-chien-thang.txt
/home/nghia/github/Music_classification_by_lyric/NCT_Crawler/Song/nguoi-di-xay-ho-ke-go-trong-tan-ft-thanh-hoa.txt
/home/nghia/github/Music_classi

In [4]:
df = pd.DataFrame(pd.Series(types)).rename(columns={0:'type'})
df['freq'] = 1
df.groupby('type').agg({'freq':'sum'}).reset_index().sort_values('freq', ascending=False)

,type,freq
22,Trữ Tình,27444
20,Thể Loại Khác,15916
6,Không Lời,15407
11,Nhạc Trẻ,14964
24,Âu Mỹ khác,6388
0,Beat,6138
13,Pop,6128
14,R&B/Hip Hop/Rap,3388
3,Cách Mạng,2321
8,Nhạc Hoa,2112
